## Real-time VLF data processor
### Vojtěch Laitl 2016
#### Ionozor Measuring Network - VLF monitors data analysis and graph processing
This is a iPython Jupyter notebook script made for VLF monitors and their real-time data processing. The computation and string codes are based on GNU Octave kernel and the cells can be run all at once.

For the easiest way to upload the final data on the storage by the common acces, the script needs the appropriate repository implemented on the server or desktop computer.

It should contain the initial folder `/.../folder/ionozor/VLF/"observatories"/"stations"/"years/"months"/`. From the location `/.../folder/`, the scripts themselves are up to be operated. The subfolders need to be written in accordance with the measuring stations and their names.

The initial environment is opened by the following repository which is based on direct requests to `space.astro.cz/ionozor/`.

In [ ]:
path0 = pwd
[YY MM DD] = datevec(date)
DD = DD
if MM < 10
m = 0
else
m = []
endif
if DD < 10
d = 0
else
d = []
endif
A = "space.astro.cz/ionozor/VLF/VLFOSTROV/R1"
A0 = "space.astro.cz/ionozor/VLF/HFKVARY/R1"
A1 = "space.astro.cz/ionozor/VLF/HFKVARY/R2"
A2 = "space.astro.cz/ionozor/VLF/HFKVARY/SOKOLOV-R1"
B = "/"
C = num2str(YY)
Ep = num2str(MM)
F = num2str(DD)
sid = "sid_"
format = ".h5"
%working with a standard -hdf5 friendly format
stat1 = "VLFOSTROV_"
stat2 = "HFKVARY_"
stat3 = "SOKOLOV-R1_"
meas1 = "R1_"
meas2 = "R2_"
spec = "spec"
x = "_"
Ip = num2str(m)
J = num2str(d)
G = [YY Ip MM J DD x sid stat1 meas1 spec format]
G0 = [YY Ip MM J DD x sid stat2 meas1 spec format]
G1 = [YY Ip MM J DD x sid stat2 meas2 spec format]
G2 = [YY Ip MM J DD x sid stat2 stat3 spec format]
path00 = [A0 B C B Ip Ep B J F B G]
path01 = [A B C B Ip Ep B J F B G]
path11 = [A1 B C B Ip Ep B J F B G1]
path21 = [A2 B C B Ip Ep B J F B G2]
%e.g. path01 = space.astro.cz/ionozor/VLF/VLFstation/VLFmon/2020/05/12/sid20200512.h5

In [ ]:
epsilon = 8.854e-12
e = 1.602e-19
h = 6.626e-34
c = 3e08
k_B = 1.38e-23
m_el = 9.109e-31
b = 2.898e-003
mD = 10^-10

The oncoming ones process real-time data of VLF spectra and the particle model, ionogram, 3D ionospheric model and solar radiation flow, shows the appropriate graph and saves they into a special HDF5 database.
Please, take into account that this tool is made for currently measuring stations and needs to be refreshed to work properly. If the upload is needed, please contact Vojtěch Laitl or Jakub Kákona via email or the Ionozor mailing list.

In [ ]:
data = urlread(path00);
a = "ionozor/VLF/HFKVARY/R1/"
b = num2str(YY)
c = num2str(MM)
local = [a b B Ip c]
cd(path0)
cd (local)
mkdir(num2str(DD))
cd (num2str(DD))
stept = 0.5
stepnu = 10

The time and frequency shift had been set in accordance with the iPython Jupyter notebook `VLF data processor.ipynb`.

In [ ]:
localsid = num2str(DD)
localpath = [local B localsid]
cd(localpath)
t0 = data(1,:)
stringt = (1:columns(t))
t = (t0 + stept.*(stringt-1))'
nu0 = data(:,1)
stringnu = ((1:rows(nu)))'
nu = nu0 + stepnu.*(stringnu-1)
L0 = data(2:,2:)
L = mean(L0,'g')
%expecting the file format after reading by Octave as [time~0.5s frequency~10Hz relative intensity~0.001dB]
dt = [0 [diff(t)]]
L1 = 0.1.*L
X = [t t]
Y = [L L]
Z = [L (-1/4.*((L./10).^-4))]
nd = -1/4.*L1.^-4
E_k = - (36*pi)^-2.*epsilon^-4/3.*nd/e^-4
p = [0; [diff(E_k)]]
nu_Delta = p.*c./h
nu = nu + nu_Delta
n = (nu.^2*m_el*epsilon)/e^2 - 1/4.*L1.^-4
dn = [0; [diff(n)]]
n0 = (nu-nu_Delta).^2*epsilon*m_el/e^2
N = n-n0
B = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*B./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*B./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((B.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
I0 = -26.74
R = 1.496*10^11
Is = exp(L./10)
fce = Is.*R.^2./I0
alpha = acos(fce)
x = abs(H.*tan(alpha))
lambda_De =  3.10399215e-05
N_D = 4/3*pi.*lambda_De.*n
p = h0.*(mD-1)
p1 = H.*(1-mD) + pi
n_e = 10.*(exp((2.*N_D./1000000.*(p1-p) + B)./B) + 4.*H)
T_EL = 3^2/3.*e^2.*n_e.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
lambda = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_EL)
dT = 10.*abs(log(b./lambda))
T = exp(dT./10)
nS = (16.*epsilon^3.*k_B^3.*pi^2.*lambda_De.^1.5.*T)./(9.*e^6.*N_D.^2)
Lambda = b./T
H1 = [H H]
N_H = [dn dn]
A = [x -x]
mesh(X,Z,Y)
xlabel"Time [hrs.]"
ylabel"L [dB]"
zlabel"dn/dL [1/m3]"
print 'Spectra.png','-dpng','-landscape','-FArial:18'
close
VLF_spectrum = [t nu L]
save -hdf5 VLF_spectrum.h5 VLF_spectrum
plot3(X,N_H,H1)
xlabel"Time [hrs.]"
ylabel"dn(t,H,L) [1/m3]"
zlabel"Height [km]"
print 'Ionogram.png','-dpng','-landscape','-FArial:18'
close
ionogram= [t dn h]
save -hdf5 ionogram.h5 ionogram
mesh(X,A,H1)
xlabel"Time interval [hrs.]"
ylabel"Horizontal flow [km]"
zlabel"Height [km]"
print 'Ionosphere.png','-dpng','-landscape','-FArial:18'
close
drift = [t x h]
save -hdf5 drift.h5 drift
plot(t, (nS./10^6))
xlabel"Time [hrs.]"
ylabel"Electron density [1/cm3]"
print 'Sun.png','-dpng','-landscape','-FArial:18'
close
sun = [t nS]
save -hdf5 sun.h5 sun

In [ ]:
data = urlread(path01);
a = "ionozor/VLF/VLFOSTROV/R1/"
b = num2str(YY)
c = num2str(MM)
local = [a b B Ip c]
cd(path0)
cd (local)
mkdir(num2str(DD))
cd (num2str(DD))
stept = 0.5
stepnu = 10

In [ ]:
localsid = num2str(DD)
localpath = [local B localsid]
cd(localpath)
t0 = data(1,:)
stringt = (1:columns(t))
t = (t0 + stept.*(stringt-1))'
nu0 = data(:,1)
stringnu = ((1:rows(nu)))'
nu = nu0 + stepnu.*(stringnu-1)
L0 = data(2:,2:)
L = mean(L0,'g')
%expecting the file format after reading by Octave as [time~0.5s frequency~10Hz relative intensity~0.001dB]
dt = [0 [diff(t)]]
L1 = 0.1.*L
X = [t t]
Y = [L L]
Z = [L (-1/4.*((L./10).^-4))]
nd = -1/4.*L1.^-4
E_k = - (36*pi)^-2.*epsilon^-4/3.*nd/e^-4
p = [0; [diff(E_k)]]
nu_Delta = p.*c./h
nu = nu + nu_Delta
n = (nu.^2*m_el*epsilon)/e^2 - 1/4.*L1.^-4
dn = [0; [diff(n)]]
n0 = (nu-nu_Delta).^2*epsilon*m_el/e^2
N = n-n0
B = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*B./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*B./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((B.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
I0 = -26.74
R = 1.496*10^11
Is = exp(L./10)
fce = Is.*R.^2./I0
alpha = acos(fce)
x = abs(H.*tan(alpha))
lambda_De =  3.10399215e-05
N_D = 4/3*pi.*lambda_De.*n
p = h0.*(mD-1)
p1 = H.*(1-mD) + pi
n_e = 10.*(exp((2.*N_D./1000000.*(p1-p) + B)./B) + 4.*H)
T_EL = 3^2/3.*e^2.*n_e.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
lambda = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_EL)
dT = 10.*abs(log(b./lambda))
T = exp(dT./10)
nS = (16.*epsilon^3.*k_B^3.*pi^2.*lambda_De.^1.5.*T)./(9.*e^6.*N_D.^2)
Lambda = b./T
H1 = [H H]
N_H = [dn dn]
A = [x -x]
mesh(X,Z,Y)
xlabel"Time [hrs.]"
ylabel"L [dB]"
zlabel"dn/dL [1/m3]"
print 'Spectra.png','-dpng','-landscape','-FArial:18'
close
VLF_spectrum = [t nu L]
save -hdf5 VLF_spectrum.h5 VLF_spectrum
plot3(X,N_H,H1)
xlabel"Time [hrs.]"
ylabel"dn(t,H,L) [1/m3]"
zlabel"Height [km]"
print 'Ionogram.png','-dpng','-landscape','-FArial:18'
close
ionogram= [t dn h]
save -hdf5 ionogram.h5 ionogram
mesh(X,A,H1)
xlabel"Time interval [hrs.]"
ylabel"Horizontal flow [km]"
zlabel"Height [km]"
print 'Ionosphere.png','-dpng','-landscape','-FArial:18'
close
drift = [t x h]
save -hdf5 drift.h5 drift
plot(t, (nS./10^6))
xlabel"Time [hrs.]"
ylabel"Electron density [1/cm3]"
print 'Sun.png','-dpng','-landscape','-FArial:18'
close
sun = [t nS]
save -hdf5 sun.h5 sun

In [ ]:
data = urlread(path11);
a = "ionozor/VLF/HFKVARY/R2/"
b = num2str(YY)
c = num2str(MM)
local = [a b B Ip c]
cd(path0)
cd (local)
mkdir(num2str(DD))
cd (num2str(DD))
cd(path0)
stept = 0.5
stepnu = 10

In [ ]:
localsid = num2str(DD)
localpath = [local B localsid]
cd(localpath)
t0 = data(1,:)
stringt = (1:columns(t))
t = (t0 + stept.*(stringt-1))'
nu0 = data(:,1)
stringnu = ((1:rows(nu)))'
nu = nu0 + stepnu.*(stringnu-1)
L0 = data(2:,2:)
L = mean(L0,'g')
%expecting the file format after reading by Octave as [time~0.5s frequency~10Hz relative intensity~0.001dB]
dt = [0 [diff(t)]]
L1 = 0.1.*L
X = [t t]
Y = [L L]
Z = [L (-1/4.*((L./10).^-4))]
nd = -1/4.*L1.^-4
E_k = - (36*pi)^-2.*epsilon^-4/3.*nd/e^-4
p = [0; [diff(E_k)]]
nu_Delta = p.*c./h
nu = nu + nu_Delta
n = (nu.^2*m_el*epsilon)/e^2 - 1/4.*L1.^-4
dn = [0; [diff(n)]]
n0 = (nu-nu_Delta).^2*epsilon*m_el/e^2
N = n-n0
B = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*B./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*B./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((B.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
I0 = -26.74
R = 1.496*10^11
Is = exp(L./10)
fce = Is.*R.^2./I0
alpha = acos(fce)
x = abs(H.*tan(alpha))
lambda_De =  3.10399215e-05
N_D = 4/3*pi.*lambda_De.*n
p = h0.*(mD-1)
p1 = H.*(1-mD) + pi
n_e = 10.*(exp((2.*N_D./1000000.*(p1-p) + B)./B) + 4.*H)
T_EL = 3^2/3.*e^2.*n_e.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
lambda = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_EL)
dT = 10.*abs(log(b./lambda))
T = exp(dT./10)
nS = (16.*epsilon^3.*k_B^3.*pi^2.*lambda_De.^1.5.*T)./(9.*e^6.*N_D.^2)
Lambda = b./T
H1 = [H H]
N_H = [dn dn]
A = [x -x]
mesh(X,Z,Y)
xlabel"Time [hrs.]"
ylabel"L [dB]"
zlabel"dn/dL [1/m3]"
print 'Spectra.png','-dpng','-landscape','-FArial:18'
close
VLF_spectrum = [t nu L]
save -hdf5 VLF_spectrum.h5 VLF_spectrum
plot3(X,N_H,H1)
xlabel"Time [hrs.]"
ylabel"dn(t,H,L) [1/m3]"
zlabel"Height [km]"
print 'Ionogram.png','-dpng','-landscape','-FArial:18'
close
ionogram= [t dn h]
save -hdf5 ionogram.h5 ionogram
mesh(X,A,H1)
xlabel"Time interval [hrs.]"
ylabel"Horizontal flow [km]"
zlabel"Height [km]"
print 'Ionosphere.png','-dpng','-landscape','-FArial:18'
close
drift = [t x h]
save -hdf5 drift.h5 drift
plot(t, (nS./10^6))
xlabel"Time [hrs.]"
ylabel"Electron density [1/cm3]"
print 'Sun.png','-dpng','-landscape','-FArial:18'
close
sun = [t nS]
save -hdf5 sun.h5 sun

In [ ]:
data = urlread(path21);
a = "ionozor/VLF/HFKVARY/SOKOLOV-R1/"
b = num2str(YY)
c = num2str(MM)
local = [a b B Ip c]
cd (local)
mkdir(num2str(DD))
cd (num2str(DD))
cd(path0)
stept = 0.5
stepnu = 10

In [ ]:
localsid = num2str(DD)
localpath = [local B localsid]
cd(localpath)
t0 = data(1,:)
stringt = (1:columns(t))
t = (t0 + stept.*(stringt-1))'
nu0 = data(:,1)
stringnu = ((1:rows(nu)))'
nu = nu0 + stepnu.*(stringnu-1)
L0 = data(2:,2:)
L = mean(L0,'g')
%expecting the file format after reading by Octave as [time~0.5s frequency~10Hz relative intensity~0.001dB]
dt = [0 [diff(t)]]
L1 = 0.1.*L
X = [t t]
Y = [L L]
Z = [L (-1/4.*((L./10).^-4))]
nd = -1/4.*L1.^-4
E_k = - (36*pi)^-2.*epsilon^-4/3.*nd/e^-4
p = [0; [diff(E_k)]]
nu_Delta = p.*c./h
nu = nu + nu_Delta
n = (nu.^2*m_el*epsilon)/e^2 - 1/4.*L1.^-4
dn = [0; [diff(n)]]
n0 = (nu-nu_Delta).^2*epsilon*m_el/e^2
N = n-n0
B = 1+(0.01.*L)
mD = 10^-10
h0 = abs(pi*i.*B./(2+2.*mD))
h0_t = (10.*h0).^2./10
f_n = dn.*B./(N.*(1-mD))
f_n2 = f_n.^2
f_ln = log(1/2.*f_n-((f_n2.^2 - 4).^1/2))
h_t = abs((B.*f_ln + 2.*pi.*i)./(1-mD))
H = 10.*(h0_t + h_t)
I0 = -26.74
R = 1.496*10^11
Is = exp(L./10)
fce = Is.*R.^2./I0
alpha = acos(fce)
x = abs(H.*tan(alpha))
lambda_De =  3.10399215e-05
N_D = 4/3*pi.*lambda_De.*n
p = h0.*(mD-1)
p1 = H.*(1-mD) + pi
n_e = 10.*(exp((2.*N_D./1000000.*(p1-p) + B)./B) + 4.*H)
T_EL = 3^2/3.*e^2.*n_e.^1/3.*N_D.^2/3./(epsilon*k_B*(4*pi)^2/3).*lambda_De.^1.2
lambda = (h*c.*lambda_De.^(N_D./100))./(k_B.*T_EL)
dT = 10.*abs(log(b./lambda))
T = exp(dT./10)
nS = (16.*epsilon^3.*k_B^3.*pi^2.*lambda_De.^1.5.*T)./(9.*e^6.*N_D.^2)
Lambda = b./T
H1 = [H H]
N_H = [dn dn]
A = [x -x]
mesh(X,Z,Y)
xlabel"Time [hrs.]"
ylabel"L [dB]"
zlabel"dn/dL [1/m3]"
print 'Spectra.png','-dpng','-landscape','-FArial:18'
close
VLF_spectrum = [t nu L]
save -hdf5 VLF_spectrum.h5 VLF_spectrum
plot3(X,N_H,H1)
xlabel"Time [hrs.]"
ylabel"dn(t,H,L) [1/m3]"
zlabel"Height [km]"
print 'Ionogram.png','-dpng','-landscape','-FArial:18'
close
ionogram= [t dn h]
save -hdf5 ionogram.h5 ionogram
mesh(X,A,H1)
xlabel"Time interval [hrs.]"
ylabel"Horizontal flow [km]"
zlabel"Height [km]"
print 'Ionosphere.png','-dpng','-landscape','-FArial:18'
close
drift = [t x h]
save -hdf5 drift.h5 drift
plot(t, (nS./10^6))
xlabel"Time [hrs.]"
ylabel"Electron density [1/cm3]"
print 'Sun.png','-dpng','-landscape','-FArial:18'
close
sun = [t nS]
save -hdf5 sun.h5 sun

After being processed, the graphs will look like following examples (included from the station R1@VLFOSTROV.space.astro.cz):
##### VLF spectra and particle model
<img src="http://extensa.emer.cz/hvezdarna/live/OSTROV/Measurement.jpg" />

##### Ionogram
<img src="http://extensa.emer.cz/hvezdarna/live/OSTROV/Response.jpg" />

##### 3D ionospheric model
<img src="http://extensa.emer.cz/hvezdarna/live/OSTROV/Ionosphere.jpg" />

##### Solar radiation flow 
<img src="http://extensa.emer.cz/hvezdarna/live/OSTROV/Sun.jpg" />.